In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
data_name = "wiki_small"

In [3]:
# with open('data/%s/hierarchy.txt' % data_name, 'w') as f1:
#     with open('data/%s/%s.ht' % (data_name,data_name)) as f:
#         for l in f:
#             split = l.strip().split(':')
#             p = split[0]
#             c = split[1].split(',')
#             for cc in c:
#                 f1.write('%s %s\n'% (p,cc))

In [4]:
# hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
#                 '%s/hierarchy.txt' % data_name)
# hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
#                    parent_of, all_name, name_to_index, level)

In [5]:
# for mode in ['train', 'test']:
#     for i in range(1,6):
#         datas, labels = preparation.import_data('%s/folds/%s_fold%d.dat.%s' % (data_name, data_name, i, mode))
#         new_labels = preparation.map_index_of_label('%s/hierarchy.pickle' % data_name, labels)
#         if mode == 'train':
#             train_data, validate_data, train_target, validate_target = train_test_split(
#                 datas, new_labels, test_size=0.1, random_state=12345)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), train_data, train_target)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, "validate"), validate_data, validate_target)
#         else:
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), datas, new_labels)

In [6]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/new_hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

# Real Flow 

In [7]:
from data import Dataset
from embedding import Doc2Vec, NoTag_Doc2Vec
from assemble_classifier import AssemblePredicted, AssemblePredictedHidden, AssemblePredictedHiddenAdd
import shutil
import os
%load_ext autoreload
%autoreload 2

In [8]:
fold = 1

In [9]:
if os.path.isdir('data/%s/output' % data_name):
    shutil.rmtree('data/%s/output' % data_name)

In [10]:
dataset_train = Dataset(data_name, fold, "train")
dataset_validate = Dataset(data_name, fold, "validate")
dataset_test = Dataset(data_name, fold, "test")

In [ ]:
dataset_train.delete_label_more(level[-1])
dataset_validate.delete_label_more(level[-1])
dataset_test.delete_label_more(level[-1])

In [ ]:
%%time
doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=150, epoch=1500, batch_size=10000)
doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels, early_stopping=True)
# doc2vec.load_model("best_now/doc2vec_wiki_small/2520.model")

Doc2Vec by Gensim
6.25
Epoch: 30 Similar: 0.08780
Epoch: 60 Similar: 0.06888
Epoch: 90 Similar: 0.06736
Epoch: 120 Similar: 0.07097
Epoch: 150 Similar: 0.07884
Epoch: 180 Similar: 0.08628
Epoch: 210 Similar: 0.09187
Epoch: 240 Similar: 0.09836
Epoch: 270 Similar: 0.10278
Epoch: 300 Similar: 0.10564
Epoch: 330 Similar: 0.10937
Epoch: 360 Similar: 0.11261
Epoch: 390 Similar: 0.11640
Epoch: 420 Similar: 0.11780
Epoch: 450 Similar: 0.12136
Epoch: 480 Similar: 0.12303
Epoch: 510 Similar: 0.12466
Epoch: 540 Similar: 0.12625
Epoch: 570 Similar: 0.12913
Epoch: 600 Similar: 0.13034
Epoch: 630 Similar: 0.13256
Epoch: 660 Similar: 0.13340
Epoch: 690 Similar: 0.13447


In [ ]:
dataset_train.change_to_Doc2Vec(doc2vec)
dataset_validate.change_to_Doc2Vec(doc2vec)
dataset_test.change_to_Doc2Vec(doc2vec)

In [ ]:
model =  AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=3000, stopping_time=400, batch_size=65536, hidden_size=[100,562,2110,3768], target_hidden_size=[60,60,60], use_dropout=True, start_level=0)
# model = AssembleNoLabel(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, batch_size=100, hidden_size=[300,1500,1500,1500], use_dropout=False, start_level=0, end_level=100)
# model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,2000,2500], use_dropout=True, start_level=0, end_level=100)

In [ ]:
%%time
model.train()

In [ ]:
model.tuning_threshold()

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("train", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("validate", correction=False)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))

In [ ]:
f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
for level, (macro, micro) in enumerate(f1_each):
    print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))